# Import material footprint as updated by [Pauliuk 2022](https://freidok.uni-freiburg.de/data/226265)

Data link: https://zenodo.org/record/6470355#.YnJ13ZJBzFw

Data file is 'mf_lcia_method_ei_3_8_export.zip'

In [1]:
import bw2data as bd
import bw2io as bi
from bw2io.strategies import link_iterable_by_fields
from bw2io.importers.json_ld_lcia import JSONLDLCIAImporter
import pprint
from functools import partial

In [2]:
bd.projects.set_current("Material Footprint")

In [3]:
bi.bw2setup()

Biosphere database already present!!! No setup is needed


In [4]:
bio = bd.Database("biosphere3")

In [5]:
if 'ecoinvent' not in bd.databases:
    fp = '/Users/cmutel/Documents/LCA/Ecoinvent/3.8/cutoff/datasets'
    ei = bi.SingleOutputEcospold2Importer(fp, 'ecoinvent')
    ei.apply_strategies()
    assert ei.all_linked
    ei.write_database()

In [6]:
def set_categories_and_name(data):
    for method in data:
        for exc in method['exchanges']:
            exc['categories'] = tuple(exc['flow']['categoryPath'][1:])
            exc['name'] = exc['flow']['name']
            
    return data

In [7]:
def fix_natural_resource_category(data):
    for method in data:
        for exc in method['exchanges']:
            if exc['categories'][0] == 'Resource':
                exc['categories'] = ('natural resource', exc['categories'][1])
                
    return data

In [8]:
def add_in_ground_if_helpful(data):
    biosphere = {o['name'] 
                 for o in bd.Database(bd.config.biosphere) 
                 if o['categories'][0] == 'natural resource'}
    
    for method in data:
        for cf in method['exchanges']:
            if not cf.get('input'):  # Skip already linked CFs
                if cf['name'] not in biosphere and cf['name'] + ", in ground" in biosphere:
                    cf['name'] += ", in ground"
    
    return data

In [9]:
def add_in_mixed_ore_if_helpful(data):
    biosphere = {o['name'] 
                 for o in bd.Database(bd.config.biosphere) 
                 if o['categories'][0] == 'natural resource'}
    
    for method in data:
        for cf in method['exchanges']:
            if "%" in cf['name'] and not cf.get('input'):  # Skip already linked CFs
                revised = cf['name'][:cf['name'].index("%") + 1] + ", in mixed ore, in ground"
                if cf['name'] not in biosphere and revised in biosphere:
                    cf['name'] = revised
    
    return data

In [10]:
js = JSONLDLCIAImporter("/Users/cmutel/Downloads/mf_lcia_method_ei_3_8_export/")
js.strategies.append(set_categories_and_name)
js.strategies.append(fix_natural_resource_category)
js.strategies.append(add_in_ground_if_helpful)
js.strategies.append(add_in_mixed_ore_if_helpful)
js.apply_strategies()

Applying strategy: json_ld_lcia_add_method_metadata
Applying strategy: json_ld_lcia_convert_to_list
Applying strategy: json_ld_lcia_set_method_metadata
Applying strategy: json_ld_lcia_reformat_cfs_as_exchanges
Applying strategy: normalize_units
Applying strategy: set_categories_and_name
Applying strategy: fix_natural_resource_category
Applying strategy: add_in_ground_if_helpful
Applying strategy: add_in_mixed_ore_if_helpful
Applied 9 strategies in 0.31 seconds


In [11]:
js.match_database('biosphere3')

Applying strategy: link_iterable_by_fields


In [12]:
js.apply_strategy(partial(link_iterable_by_fields, other=filter(lambda x: x['categories'][0] == 'natural resource', bio), fields=['name', 'unit']))

Applying strategy: link_iterable_by_fields


In [13]:
js.statistics()

16 methods
1299 cfs
6 unlinked cfs


(16, 1299, 6)

There is only one remaining unlinked CF, for nickel:

In [14]:
for obj in js.unlinked:
    pprint.pprint(obj)

{'@type': 'ImpactFactor',
 'amount': 63,
 'categories': ('natural resource', 'in ground'),
 'flow': {'@id': '0d7f8b87-12f4-4e83-a5a2-854e2f2b47de',
          '@type': 'Flow',
          'categoryPath': ['Elementary flows', 'Resource', 'in ground'],
          'flowType': 'ELEMENTARY_FLOW',
          'name': 'Nickel, Ni 2.3E+0%, Pt 2.5E-4%, Pd 7.3E-4%, Rh 2.0E-5%, Cu '
                  '3.2E+0% in ore',
          'refUnit': 'kg'},
 'flowProperty': {'@id': '93a60a56-a3c8-11da-a746-0800200b9a66',
                  '@type': 'FlowProperty',
                  'categoryPath': ['Technical flow properties'],
                  'name': 'Mass'},
 'name': 'Nickel, Ni 2.3E+0%, Pt 2.5E-4%, Pd 7.3E-4%, Rh 2.0E-5%, Cu 3.2E+0% '
         'in ore',
 'unit': 'kilogram'}


But this is well covered by other nickel ores:

In [15]:
{cf['name'] for method in js.data for cf in method['exchanges'] if cf['name'].startswith('Nickel')} 

{'Nickel, 1.13% in sulfide, Ni 0.76% and Cu 0.76% in crude ore, in ground',
 'Nickel, 1.98% in silicates, 1.04% in crude ore, in ground',
 'Nickel, Ni 2.3E+0%, Pt 2.5E-4%, Pd 7.3E-4%, Rh 2.0E-5%, Cu 3.2E+0% in ore',
 'Nickel, Ni 2.5E+0%, in mixed ore, in ground',
 'Nickel, in ground'}

And this nickel ore is not in our flow list:

In [16]:
{x['name'] for x in bio if x['name'].startswith('Nickel')}

{'Nickel',
 'Nickel, 1.13% in sulfide, Ni 0.76% and Cu 0.76% in crude ore, in ground',
 'Nickel, 1.98% in silicates, 1.04% in crude ore, in ground',
 'Nickel, Ni 2.5E+0%, in mixed ore, in ground',
 'Nickel, in ground',
 'Nickel, ion'}

In [17]:
js.drop_unlinked(True)

Applying strategy: drop_unlinked_cfs
Applied 1 strategies in 0.00 seconds


In [65]:
js.write_methods()

Wrote 16 LCIA methods with 1293 characterization factors
